## Итоговый проект по курсу от Megafon

### Часть 1. Исследование и предварительная подготовка данных

### Легенда

У нас появился запрос из отдела продаж и маркетинга. Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам. При этом разным пользователям интересны разные услуги. Поэтому необходимо построить алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.

### Данные

В качестве исходных данных вам будет доступна информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить. Отдельным набором данных будет являться нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени. Данные train и test разбиты по периодам – на train доступно 4 месяцев, а на test отложен последующий месяц.

### В итоге, в качестве входных данных будут представлены:

data_train.csv: id, vas_id, buy_time, target. features.csv.zip

### Тестовый набор:

data_test.csv: id, vas_id, buy_time.   target - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно. buy_time - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime. id - идентификатор абонента vas_id - подключаемая услуга Примечание: Размер файла features.csv в распакованном виде весит 20 гб, для работы с ним можно воспользоваться pandas.read_csv, либо можно воспользоваться библиотекой Dask.

### Метрика

Скоринг будет осуществляться функцией f1, невзвешенным образом, как например делает функция sklearn.metrics.f1_score(…, average=’macro’).

### Импоритруем библиотеки

In [1]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
from datetime import datetime

from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import catboost as catb
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

def convert_timestamp_to_datetime(dataframe, colname):
    dataframe[colname] = dataframe[colname].apply(lambda timestamp: datetime.fromtimestamp(timestamp))
    return dataframe

### Зададим константы путей к данным

In [2]:
DATA_TRAIN_PATH = 'data_train.csv'
DATA_TEST_PATH = 'data_test.csv'
FEATURES_PATH = 'features.csv'

### Загружаем данные

In [3]:
data_train = pd.read_csv(DATA_TRAIN_PATH)
data_test = pd.read_csv(DATA_TEST_PATH)

In [4]:
%%time
data_features = dd.read_csv(FEATURES_PATH, sep='\t')

Wall time: 155 ms


### Посмотрим на содержание данных

In [5]:
data_train.head(2)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0


In [6]:
data_test.head(2)

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000


In [7]:
data_features.head(2)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0


In [8]:
data_train.shape[0]

831653

In [9]:
data_test.shape[0]

71231

In [10]:
data_test['id'].value_counts()

1374678    3
44545      3
3240429    3
2615828    3
1210522    3
          ..
1790395    1
1441300    1
1790399    1
2511623    1
988236     1
Name: id, Length: 70152, dtype: int64

### Исследование данных

In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  831653 non-null  int64  
 1   id          831653 non-null  int64  
 2   vas_id      831653 non-null  float64
 3   buy_time    831653 non-null  int64  
 4   target      831653 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 31.7 MB


In [12]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71231 entries, 0 to 71230
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  71231 non-null  int64  
 1   id          71231 non-null  int64  
 2   vas_id      71231 non-null  float64
 3   buy_time    71231 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 2.2 MB


### Посмотрим на количество классов и их соотношение в "target":

In [13]:
data_train['target'].value_counts()

0.0    771467
1.0     60186
Name: target, dtype: int64

**У нас два класса: 1 и 0 и наблюдается их явный дисбаланс, что в последствии необходимо учесть.**

**Удалим признак "Unnamed: 0", который для нас бесполезен, так как дублирует "id".**

In [14]:
data_train = data_train.drop(columns=['Unnamed: 0'])
data_test = data_test.drop(columns=['Unnamed: 0'])

**Проверим датасет на дублирующиеся id:**

In [15]:
data_train.duplicated(subset=['id']).value_counts()

False    806613
True      25040
dtype: int64

In [16]:
data_test.duplicated(subset=['id']).value_counts()

False    70152
True      1079
dtype: int64

**Видим, что есть повторяющиея id. Вероятно это id пользователей, у которых несколько sim-карт.**

### Просмотрим, сколько всего данных в датасете "features":

In [17]:
%%time
data_features.shape[0].compute()

Wall time: 5min 15s


4512528

In [18]:
# 4512528

In [19]:
data_features.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 256 entries, Unnamed: 0 to 252
dtypes: float64(253), int64(3)

**Количество данных в датасете достаточно велико. Отфильтруем бесполезные и оставим только признаки с id наших клиентов из train и test.**

In [20]:
def prepare_features(features, train, test):
    user_ids = np.unique(train['id'].append(test['id']))
    # Оставим id пользователей, из train и test
    features = features[features['id'].isin(user_ids)]

    # Удаляем столбец Unnamed: 0
    if 'Unnamed: 0' in features:
        features = features.drop(columns='Unnamed: 0')
    features = features.compute()
    
    # Удаляем признаки с единственным значением
    df_nunique = features.apply(lambda x: x.nunique(dropna=False))
    const = df_nunique[df_nunique == 1].index.tolist()
    features = features.drop(columns = const)
    return features

In [21]:
%%time
features = prepare_features(data_features, data_train, data_test)
features

Wall time: 5min 45s


,id,buy_time,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,250,251,252
13,2046132,1534712400,300.820029,1599.480888,286.879214,1585.013202,281.461754,1563.90821,-16.08618,654.013903,...,-977.373846,-613.770792,-25.996269,-35.630448,-295.747724,-17.832889,-0.694428,-4.175933,-0.45614,0.0
16,2050810,1540760400,-86.209971,91.820888,-84.480786,110.333202,-89.898246,89.22821,-16.08618,-65.076097,...,-977.373846,-613.770792,-23.996269,190.369552,-286.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
19,2070757,1540760400,-96.799971,-408.179112,-110.740786,-460.786798,-114.038246,-479.77179,-16.08618,-65.076097,...,-925.373846,-561.770792,-21.996269,-37.630448,-151.747724,-24.832889,0.305572,-12.175933,-0.45614,1.0
20,2071522,1544994000,-94.939971,-363.699112,-108.880786,-411.226798,-114.298246,-432.33179,-16.08618,-65.076097,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
22,2075318,1533502800,-75.639971,669.690888,-89.580786,732.343202,-94.998246,736.65821,-16.08618,782.383903,...,-501.373846,-242.770792,-25.996269,-37.630448,-167.747724,-14.832889,2.305572,-4.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,3513869,1548018000,82.370029,-123.429112,155.939214,-88.526798,150.521754,-109.63179,-16.08618,-65.076097,...,-928.373846,-570.770792,-23.996269,-37.630448,-271.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
6693,3516552,1547413200,-96.799971,-116.519112,-110.740786,-169.126798,-116.158246,-190.23179,-16.08618,226.583903,...,-975.373846,-613.770792,-25.996269,-37.630448,-5.747724,-24.832889,-0.694428,-12.175933,-0.45614,0.0
6697,3517434,1548018000,-96.799971,-284.349112,-100.740786,-274.796798,-106.158246,-295.90179,-16.08618,-65.076097,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
6705,3519714,1546808400,167.280029,110.140888,153.339214,57.533202,147.921754,36.42821,-2.00618,-50.996097,...,-977.373846,-613.770792,-14.996269,7.369552,-180.747724,-19.832889,0.305572,-12.175933,-0.45614,0.0


### Объединим features с train и test

In [22]:
%%time
df_train_with_features = data_train.merge(features, on='id', how = 'left')

Wall time: 6.33 s


In [23]:
%%time
df_test_with_features = data_test.merge(features, on='id', how = 'left')

Wall time: 2.62 s


In [24]:
df_train_with_features

,id,vas_id,buy_time_x,target,buy_time_y,0,1,2,3,4,...,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,0.0,1541970000,-31.559971,327.360888,-45.500786,274.753202,-50.918246,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1454121,4.0,1531688400,0.0,1531083600,547.270029,238.430888,533.329214,274.803202,527.911754,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2458816,1.0,1534107600,0.0,1543179600,-92.139971,-95.469112,-106.080786,-139.596798,-111.498246,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3535012,5.0,1535922000,0.0,1533502800,54.880029,12.970888,54.079214,-9.116798,48.661754,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,1693214,1.0,1535922000,0.0,1543179600,45.160029,295.240888,64.679214,344.283202,59.261754,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860047,3812226,2.0,1546203600,0.0,1532898000,29.750029,6.200888,24.279214,-37.936798,18.861754,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
860048,2480469,2.0,1546203600,0.0,1532293200,-89.179971,-56.499112,-103.120786,-109.106798,-108.538246,...,-66.373846,-321.770792,-25.996269,62.369552,1167.252276,41.167111,-0.694428,54.824067,-0.45614,0.0
860049,158236,2.0,1546203600,0.0,1538946000,-96.799971,62.140888,-110.740786,9.533202,-116.158246,...,-683.373846,-470.770792,-25.996269,-37.630448,99.252276,178.167111,-0.694428,191.824067,-0.45614,0.0
860050,1825525,2.0,1546203600,0.0,1533502800,-96.799971,-81.919112,-110.740786,-128.596798,-116.158246,...,1018.626154,1367.229208,-24.996269,-35.630448,-237.747724,-21.832889,-0.694428,-8.175933,-0.45614,1.0


In [25]:
df_test_with_features

,id,vas_id,buy_time_x,buy_time_y,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,3130519,2.0,1548018000,1536526800,-62.899971,-374.279112,-72.600786,-418.406798,-78.018246,-439.51179,...,-970.373846,-613.770792,-25.996269,-37.630448,-258.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,2000860,4.0,1548018000,1532293200,-96.799971,100.290888,-110.740786,140.903202,-116.158246,119.79821,...,-977.373846,-613.770792,-25.996269,-37.630448,-254.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
2,1099444,2.0,1546808400,1541365200,-81.969971,-390.729112,-95.910786,-443.336798,-101.328246,-464.44179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
3,1343255,5.0,1547413200,1536526800,259.130029,-52.249112,245.189214,-104.856798,239.771754,-125.96179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4,1277040,2.0,1546808400,1532293200,331.170029,590.890888,317.229214,538.283202,311.811754,522.25821,...,-651.373846,-612.770792,1.003731,-36.630448,38.252276,-12.832889,-0.694428,-12.175933,12.54386,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73661,2502453,5.0,1548018000,1534712400,-96.799971,-408.179112,-110.740786,-460.786798,-116.158246,-481.89179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
73662,1693213,2.0,1548018000,1541970000,-89.689971,-355.809112,-103.630786,-408.416798,-109.048246,-429.52179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
73663,1891350,2.0,1548018000,1545598800,-96.799971,-281.059112,-110.740786,-333.666798,-116.158246,-354.77179,...,1640.626154,1927.229208,-25.996269,-32.630448,-33.747724,-18.832889,-0.694428,-10.175933,-0.45614,0.0
73664,2437172,2.0,1548018000,1532293200,115.060029,-87.339112,101.119214,-127.236798,95.701754,-148.34179,...,-977.373846,-613.770792,-25.996269,-37.630448,-280.747724,119.167111,-0.694428,132.824067,-0.45614,0.0


In [26]:
df_test_with_features.duplicated(subset=['id']).value_counts()

False    70152
True      3514
dtype: int64

In [27]:
# %%time
# df_train_with_features.to_csv(DATA_TRAIN_WITH_FEATURES_PATH)

In [28]:
# %%time
# df_test_with_features.to_csv(DATA_TEST_WITH_FEATURES_PATH)

In [29]:
# features.to_csv(PREPARE_FEATURES_PATH)

### Разделим датасет на train и test.

In [30]:
X = df_train_with_features.drop('target', axis=1)
y = df_train_with_features.target
X_actual = df_test_with_features

In [31]:
X.head(2)

,id,vas_id,buy_time_x,buy_time_y,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,1541970000,-31.559971,327.360888,-45.500786,274.753202,-50.918246,253.64821,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1454121,4.0,1531688400,1531083600,547.270029,238.430888,533.329214,274.803202,527.911754,253.69821,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0


In [32]:
y.head(2)

0    0.0
1    0.0
Name: target, dtype: float64

In [33]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Wall time: 4.59 s


### Часть 2. Обучение модели и оценка результата

**Напишем функцию для удобного вывода результатов:**

In [34]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

**В качестве модели возьмем CatBoostClassifier. Модель хорошо работает с пропусками и категориальными данными.**

**У нас наблюдается сильный дисбаланс классов, поэтому применим параметр auto_class_weights='Balanced'.**

In [35]:
%%time
model = catb.CatBoostClassifier(auto_class_weights='Balanced',
                                early_stopping_rounds=20,
                                silent=True, 
                                random_state=42)
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_test_pred_probs = model.predict_proba(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

TRAIN

              precision    recall  f1-score   support

         0.0       0.99      0.87      0.92    638307
         1.0       0.35      0.91      0.50     49734

    accuracy                           0.87    688041
   macro avg       0.67      0.89      0.71    688041
weighted avg       0.95      0.87      0.89    688041

TEST

              precision    recall  f1-score   support

         0.0       0.99      0.87      0.93    159538
         1.0       0.35      0.91      0.51     12473

    accuracy                           0.87    172011
   macro avg       0.67      0.89      0.72    172011
weighted avg       0.95      0.87      0.89    172011

CONFUSION MATRIX

col_0      0.0    1.0
target               
0.0     138315  21223
1.0       1086  11387
Wall time: 1min 47s


### Выведем скоринг модели отдельной строкой:

In [36]:
f1_score(
  y_test,
  y_test_pred,
  average='macro'
)

0.7152649447436665

### Посмотрим какие признаки для модели оказались наиболее полезными: 

In [37]:
model.feature_importances_

array([8.80415510e-01, 5.12705413e+01, 4.50679295e+01, 1.98113412e-02,
       7.58608404e-03, 1.16884456e-02, 4.83623124e-03, 1.79960691e-02,
       2.43652500e-02, 1.51878378e-02, 1.93789182e-02, 0.00000000e+00,
       2.28037401e-03, 7.25873612e-03, 2.82091543e-03, 1.60922178e-03,
       5.10111113e-03, 7.62123479e-03, 7.32523292e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.45995896e-03, 1.94030214e-02,
       3.80299682e-03, 2.18462349e-02, 4.41126029e-03, 0.00000000e+00,
       0.00000000e+00, 2.17727819e-02, 0.00000000e+00, 2.87358416e-03,
       5.63849501e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.31449002e-03, 0.00000000e+00,
       3.68257724e-02, 2.19319476e-02, 6.99583155e-03, 1.51177181e-02,
       1.93780048e-02, 1.77656374e-02, 4.35918397e-03, 4.43375756e-02,
       2.90906466e-02, 2.95044296e-03, 1.11938742e-02, 1.04378562e-02,
       1.56028517e-02, 0.00000000e+00, 1.09135496e-02, 1.99440983e-02,
      

In [38]:
feature_importances = pd.DataFrame(zip(X_train.columns, 
                                   model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

# feature_importances.sort_values(by='importance', ascending=, inplace=True)

In [39]:
feature_importances.head()

,feature_name,importance
0,id,0.880416
1,vas_id,51.270541
2,buy_time_x,45.067929
3,buy_time_y,0.019811
4,0,0.007586


In [40]:
feature_importances.sort_values(by='importance', ascending=False, inplace=True)

In [41]:
feature_importances.head()

,feature_name,importance
1,vas_id,51.270541
2,buy_time_x,45.067929
0,id,0.880416
116,115,0.060481
56,52,0.059555


In [42]:
model.predict_proba(X_test)

array([[0.905905  , 0.094095  ],
       [0.88982179, 0.11017821],
       [0.9818836 , 0.0181164 ],
       ...,
       [0.99869639, 0.00130361],
       [0.80539099, 0.19460901],
       [0.94597768, 0.05402232]])

In [43]:
model.predict(X_test)

array([0., 0., 0., ..., 0., 0., 0.])

### Сохраним готовую модель в файл:

In [44]:
import pickle

In [45]:
with open('./model.pkl', 'wb') as file:
    pickle.dump(model, file)

### Часть 3. Определим вероятность подключения услуги пользователями 

### Загрузим модель из файла, дадим ей другое название и проверим её работу на тестовой выборке:

In [46]:
with open('./model.pkl', 'rb') as file:
    my_model = pickle.load(file)
    
my_model

In [47]:
# import sklearn
# print(sklearn.__version__)

In [48]:
# pip freeze

In [49]:
y_actual = my_model.predict(X_actual)

### Сохраним полученные предсказания в таблицу

In [50]:
answers_test = X_actual[['id', 'vas_id', 'buy_time_x']].copy()
answers_test['target'] = y_actual

In [51]:
answers_test.head()

,id,vas_id,buy_time_x,target
0,3130519,2.0,1548018000,0.0
1,2000860,4.0,1548018000,1.0
2,1099444,2.0,1546808400,0.0
3,1343255,5.0,1547413200,0.0
4,1277040,2.0,1546808400,0.0


### Результат сохраним в файл

In [52]:
answers_test.to_csv('answers_test.csv')